# Self Attension (Attension Mechanism)

REF: https://www.youtube.com/watch?v=QCJQG4DuHT0&ab_channel=CodeEmporium

- Self-attention is a technique in machine learning that allows models to weigh the importance of different parts of an input sequence.
- Self-Attension is solve the problem of RNN (Recurrent Neural Network) in case RNN is feed in model one by one word and can remember the weight of the last word that just feeded to model.

INPUT: Word Vector
OUTPUT: Word Vector with Self Attension included
PROCESS:

- Use Recurrent Neural Network
- This process is come after Positional Encoding to fine-grained more context
- Parameters
    - L: length of words sequence
    - d_k (Key): user's define
    - d_v (Value): user's define
- Equation of weight calculation (we will keep this for real learning later in ANN)

For a single head:
$$
\text{self attention} = softmax\bigg(\frac{Q.K^T}{\sqrt{d_k}}+M\bigg)
$$

$$
\text{new V} = \text{self attention}.V
$$ 

    - q, k, v:
        - q: input x weight_q
        - k: input x weight_k
        - v: input x weight_v
        - each q, k, v comes from Linear Transform Function (Q = W_Q * X + b_Q) | where X is input_batch
    - np.matmul(q, k.T): Matrix multiply to get every word relationship (L x L)
        - T: Transpose
    - sqrt(dk): as denominator, Divind this to minimize variant and stabilize the vector
    - V (Value): input vector
    - new V: output after calculate self-attension
    - M (Masking):
```python
mask = np.tril(np.ones((L, L)))
mask[mask == 0] = -np.infty
mask[mask == 1] = 0
```
        - Ensure to not get future word while generating new word (prevent model to cheat)
        - `required in decoder` but not require in encoder
        - Triangular Matrix of 1: to let input focus only current and the past word (1) not future words (0) when multiply
            - convert 1 to 0 and 0 to -inf: same concept but to support add (+) matrix and Softmax later
            

# Multi-head Attension

REF: https://www.youtube.com/watch?v=HQn1QKQYXVg&list=PLTl9hO2Oobd97qfWC40gOSU8C0iu0m2l4&index=2&ab_channel=CodeEmporium

- Allow Self-Attension to run in parallel
- Separate the weight in each focus self-attension head - lead to better result
- transpose each vector and send to each attension unit (head)

##### How

<img src="./images/multi_head_attension_s1.png">

- Convert each word to vector
    - e.g. 512x1
- Break down into 3 component vector (each have 512x1)
    - Query: What am I looking for
    - Key: What I can offer
    - Value: What I ACTUALLY offer
- Define attension heads number (e.g. 8) and break down each QKV into 8 piece (each 512 / 8 = 64 | 64 x 1)
- All heads feed input to `Attension Unit`
- Generate `Attension Matrix` for each head.
    - 4x4 matrix (input has 4 words)
    - Value is 0-1 (added up to 1) as probabilistic
    - To generate contextual awareness vector
- send to softmax function to normalized output in 0 - 1
- concatenate every head